In [ ]:
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import patsy

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
data = pd.read_csv('datasets/covtype.data', header=None)
data[54] = data[54].apply(lambda x: str(x))

curr_cols = list(data.columns)
curr_cols[-1] = 'target'
data.columns = curr_cols

data = data.sample(2000)

In [ ]:
Y = patsy.dmatrix('target -1', data, return_type='dataframe')
Y.head()

In [ ]:
Y.sum(axis=0)

In [ ]:
X = data[curr_cols[:-1]]
X.shape

In [ ]:
Y.shape

In [ ]:
print(X.shape[1], Y.shape[1])

In [ ]:
def logit(X):
    return 1 / (1 + tt.exp(-X))

In [ ]:
# Bayesian multi-class logistic regression to predict the target classes.
import theano.tensor as tt

with pm.Model() as model:
    # degrees_freedom = pm.HalfCauchy('degrees_freedom', 100)
    # weights = pm.StudentT('weights', 0, 100**2, degrees_freedom, shape=(X.shape[1],))
    # intercept = pm.StudentT('intercept', 0, 100**2, degrees_freedom)
    
    weights = pm.Normal('weights', 0, 100**2, shape=(X.shape[1], Y.shape[1]))
    intercept = pm.Normal('intercept', 0, 100**2)

    exponent = tt.dot(X, weights) + intercept
    p = logit(exponent)
    
    like = pm.Multinomial('likelihood', n=1, p=p, observed=Y)

In [ ]:
with model:
    trace = pm.sample(10000, start=pm.find_MAP(), step=pm.Metropolis())

In [ ]:
pm.traceplot(trace)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
with model:
    ppc_samps = pm.sample_ppc(trace, samples=100, )